In [9]:
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from rich import print
from rich.panel import Panel
from rich.console import Console
from rich.markdown import Markdown
import torch

In [ ]:

base_model = "mistralai/Mistral-Nemo-Instruct-2407"
model_name = "shikhac30/mistral_nemo_12b_medical_model"  


In [6]:
# Load the model and tokenizer from HF or local path
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)

# Build the pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Device set to use cuda:0


In [ ]:
    

# Input prompt
user_input = """Patient presents with acute onset of dyspnea and orthopnea;
CXR reveals bilateral perihilar infiltrates suggestive of cardiogenic pulmonary edema.
BNP elevated, echocardiogram pending to evaluate for systolic dysfunction."""

formatted_input = f"<s>[INST] {user_input.strip()} [/INST]"

result = pipe(formatted_input, max_new_tokens=128, do_sample=True, temperature=0.7)

output_text = result[0]["generated_text"].replace(formatted_input, "").strip()

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [10]:
console = Console()
console.print(Panel.fit(user_input, title="[bold blue]Input Prompt", border_style="blue"))
console.print(Panel.fit(output_text, title="[bold magenta]Model Output", border_style="magenta"))

╭───────────────────────────────────── Input Prompt ─────────────────────────────────────╮
│ Patient presents with acute onset of dyspnea and orthopnea;                            │
│ CXR reveals bilateral perihilar infiltrates suggestive of cardiogenic pulmonary edema. │
│ BNP elevated, echocardiogram pending to evaluate for systolic dysfunction.             │
╰────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Model Output ──────────────────────────────────────────────────╮
│ She has suddenly started having difficulty breathing and feels short of breath when lying down. Tests show      │
│ inflammation in both lungs and a high level of a certain blood marker. They're waiting for more tests to check  │
│ her heart function.                                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯